In [1]:
import torch
from torchvision.models import resnet18
import pickle
import numpy as np
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names

In [2]:
model = resnet18()

In [4]:
x_train, x_test = get_graph_node_names(model)

In [8]:
x_test = [l for l in x_test if "layer" and "conv" in l][1:]

In [12]:
dic = {key:key for key in x_test}

In [13]:
dic

{'layer1.0.conv1': 'layer1.0.conv1',
 'layer1.0.conv2': 'layer1.0.conv2',
 'layer1.1.conv1': 'layer1.1.conv1',
 'layer1.1.conv2': 'layer1.1.conv2',
 'layer2.0.conv1': 'layer2.0.conv1',
 'layer2.0.conv2': 'layer2.0.conv2',
 'layer2.1.conv1': 'layer2.1.conv1',
 'layer2.1.conv2': 'layer2.1.conv2',
 'layer3.0.conv1': 'layer3.0.conv1',
 'layer3.0.conv2': 'layer3.0.conv2',
 'layer3.1.conv1': 'layer3.1.conv1',
 'layer3.1.conv2': 'layer3.1.conv2',
 'layer4.0.conv1': 'layer4.0.conv1',
 'layer4.0.conv2': 'layer4.0.conv2',
 'layer4.1.conv1': 'layer4.1.conv1',
 'layer4.1.conv2': 'layer4.1.conv2'}

In [ ]:
xlayer = model.layer1[1].conv1
scale_factor = 2
kernel_vectors = xlayer.weight.flatten(1)
l2_distances = torch.cdist(kernel_vectors, kernel_vectors).triu().unsqueeze(0)
l2_blocks = torch.nn.functional.unfold(l2_distances, kernel_size=scale_factor, stride=scale_factor, padding=0).T
step_size = (l2_distances.shape[1] // (scale_factor )) + 1
extract_indices = torch.arange(0, l2_blocks.shape[0], step=step_size)
repulsion_loss = -l2_blocks[extract_indices].sum()

In [ ]:
l2_blocks[extract_indices].sum(dim=1)

In [ ]:
repulsion_loss

In [ ]:
scale_factor=2

In [ ]:
tot_repulsion_loss = 0
for name, param in model.named_modules():
    if "conv" in name and "layer" in name:
        kernel_vectors = param.weight.flatten(1)
        l2_distances = torch.cdist(kernel_vectors, kernel_vectors).triu().unsqueeze(0)
        l2_blocks = torch.nn.functional.unfold(l2_distances, kernel_size=scale_factor, stride=scale_factor, padding=0).T
        step_size = (l2_distances.shape[1] // (scale_factor )) + 1
        extract_indices = torch.arange(0, l2_blocks.shape[0], step=step_size)
        repulsion_loss = -l2_blocks[extract_indices].sum()
        print(repulsion_loss)
        tot_repulsion_loss+=repulsion_loss    

In [ ]:
tot_repulsion_loss*0.0005

In [ ]:

step = 2
images = torch.randn((20,10,10))
batch_size = images.shape[0]
condition = torch.randn((batch_size,)) > 0
mask = condition
binary_success = condition.to(dtype=int)
cache_size =  10
success_thres = 0.5


In [ ]:
images = torch.randn((20,10,10))
batch_size = images.shape[0]
condition = torch.randn((batch_size,)) > 0
mask = condition
binary_success = condition.to(dtype=int)
cache_size =  10
success_thres = 0.5

moving_results = torch.zeros(size=(batch_size, cache_size))
moving_results[:] = torch.nan
moving_avg = torch.zeros(size=(batch_size,))
best_moving_avg = torch.negative(torch.ones(size=(batch_size,)))

In [ ]:
cache_idx = step % cache_size
moving_results[:, cache_idx] = binary_success
moving_avg = torch.nanmean(moving_results, dim=1)

# Update images with moving_avg


In [ ]:
mask_mov_avg = moving_avg > best_moving_avg
mask = torch.logical_and(mask_mov_avg, mask)

In [ ]:
moving_results

In [ ]:
moving_avg

In [23]:
x = ["┌─","─┐", "─┬─", "└─", "─┘", "─┴─", "├─", "─┤", "─┼─" ]


In [15]:
for el in x:
    print(el)
    for ch in el:
        print(ord(ch)) 
    print("\n\n")

NameError: name 'x' is not defined

In [56]:
loss = 2.65
all_repulsion_losses = torch.tensor([-100, -90, -80, -70, -60, -50, -40, -30]).to(dtype=torch.float32)
lam = torch.tensor([i*4 for i in range(1,len(all_repulsion_losses)+1)]).to(dtype=torch.float32)
total_repulsion_loss = torch.dot(all_repulsion_losses, lam)

In [57]:
tot_repulsion_loss_c = torch.negative(total_repulsion_loss.clone())
lam_i = 1
while tot_repulsion_loss_c > loss:
    tot_repulsion_loss_c = tot_repulsion_loss_c / 10
    lam_i*=1/10


In [58]:
lam

tensor([ 4.,  8., 12., 16., 20., 24., 28., 32.])

In [59]:
lam = torch.tensor([lam[i]*lam_i for i in range(len(lam))])

In [54]:
total_repulsion_loss = torch.dot(all_repulsion_losses, lam)

In [55]:

total_repulsion_loss

tensor(-0.7680)

In [49]:
loss

2.65

In [2]:
lam = 1
rep_loss_c = rep_loss
while rep_loss_c > loss:
    lam*=1/10
    rep_loss_c= rep_loss_c / 10